# Tempus Bioinformatics Technical Challenge
The goal of this challenge is to prototype a variant annotation tool that accepts VCF files and outputs TSV files. This notebook will serve as an extended guide of the Python script.

### Getting Started
First, we import the necessary libraries. Pysam will be used to load and parse the VCF file. Requests will be used to access the Exome Aggregation Consortium (ExAC) API (http://exac.hms.harvard.edu/) in order to get data about the variants. The json package will parse data from the ExAC API.

In [899]:
from pysam import VariantFile
import requests
import json

The **get_variants** function returns a list containing the variants (rows) from the VCF file. The .fetch() function generates VariantRecord objects that allow access to attributes of each variant from the VCF header.

In [900]:
def get_variants(vcf):
    return [var for var in vcf.fetch()]

The **get_samples** function simply returns a list of the names of the samples in the VCF file.

In [901]:
def get_samples(vcf):
    return list(vcf.header.samples)

Let's load the vcf data.

In [902]:
data = VariantFile('Challenge_data.vcf')
data

The **get_records** function fetches all the variants from data. We're only keeping the first 100 rows for simplicity reasons. The first row is printed here.

In [903]:
variants = get_variants(data)
variants = variants[0:100]
print(variants[0])

1	931393	.	G	T	2.17938e-13	.	AB=0;ABP=0;AC=0;AF=0;AN=6;AO=95;CIGAR=1X;DP=4124;DPB=4124;DPRA=0.999031;EPP=9.61615;EPPR=316.776;GTI=0;LEN=1;MEANALT=1;MQM=59.7579;MQMR=65.2274;NS=2;NUMALT=1;ODDS=591.29;PAIRED=0.989474;PAIREDR=0.966741;PAO=0;PQA=0;PQR=0;PRO=0;QA=3774;QR=160284;RO=4029;RPL=51;RPP=4.13032;RPPR=101.278;RPR=44;RUN=1;SAF=40;SAP=8.15326;SAR=55;SRF=1663;SRP=269.369;SRR=2366;TYPE=snp	GT:GQ:DP:DPR:RO:QR:AO:QA	0/0/0:132.995:2063:2063,0:2063:82063:0:0	0/0/0:132.995:2061:2061,95:1966:78221:95:3774



The names of the samples can be retrieved with the **get_samples** function.

In [904]:
samples = get_samples(data)
print(samples)

['normal', 'vaf5']


### Annotation Functions
Now, we have the functions that will perform the annotations on the VCF file. The input for these functions is always the list returned from **get_variants**. Because some variants have multiple alleles, most of these functions will follow a similar pattern. There is an if statement that handles variants that only have one allele and converted to a string. There is an if statement that handles variants with multiple alleles in which all annotations are concatenated into one string, but comma separated.

The **variant_types** function iterates through the variants and gets the variant type. The variant type is taken from the VCF header:
##INFO=<ID=TYPE,Number=A,Type=String,Description="The type of allele, either snp, mnp, ins, del, or complex.">

If there is more than one variant, all types will be concatenated into one string and appended to the list. 

In [905]:
def variant_types(variants):
    var_types = []
    
    for var in variants:
        var_type = var.info['TYPE']
        
        # one type associated with variant
        if len(var_type) == 1:
            var_types.append(var_type[0])
            
        # multiple types associated with variant because there are multiple alleles
        elif len(var_type) > 1:
            var_types.append(','.join(var.info['TYPE']))
            
    return var_types

Let's run **variant_types**. Two results have been printed here. The first result is a variant that is a snp i.e. a substitution. The second result is a variant with two alleles. One allele is a deletion, the other is an insertion. As you can see, variants with more than one allele have their results concatenated into one string.

In [906]:
var_types = variant_types(variants)
print(var_types[0])
print(var_types[53])

Variant types annotated.
snp
del,ins


The **effects** function returns the consequence or effect caused by the variant. The effect caused by variant is determined from the ExAC API. If there is no record of it in ExAC, then a '.' denotes a null value. Sometimes, an allele may cause multiple effects. In this case, a different function, rank_effects, is called that determines and keeps only the most deleterious effect.

In [915]:
def effects(variants):
    effects = []
    
    for var in variants:
        chrom = str(var.chrom)
        pos = str(var.pos)
        ref = var.ref
        alts = var.alts

        # one allele associated with variant
        if len(alts) == 1:

            # ExAC API request
            var_id = f'{chrom}-{pos}-{ref}-{alts[0]}'
            request = f'http://exac.hms.harvard.edu/rest/variant/consequences/{var_id}'
            response = requests.get(request)
            text = json.loads(response.text)
            
            # No recorded effects associated with variant
            if text is None or text == {}:
                effects.append('.')
            
            # there is at least one effect associated  
            else:
                effect = list(text.keys())
                
                # only one effect is associated
                if len(effect) == 1:
                    effects.append(str(effect[0]))
                
                # more than one effects are associated, so the most deleterious effect is chosen
                elif len(effect) > 1:
                    effects.append(rank_effects(effect))

        # more than one alleles associated with variant
        if len(alts) > 1:
            effects_temp = []
            
            # iterate through each allele
            for alt in alts:

                # ExAC API request
                var_id = f'{chrom}-{pos}-{ref}-{alt}'
                request = f'http://exac.hms.harvard.edu/rest/variant/consequences/{var_id}'
                response = requests.get(request)
                text = json.loads(response.text)
                
                # No recorded effects associated with variant
                if text is None or text == {}:
                    effects_temp.append('.')
                
                # there is at least one effect associated
                else:
                    effect = list(text.keys())
                    
                    # only one effect is associated
                    if len(effect) == 1:
                        effects_temp.append(str(effect[0])) 
                    
                    # more than one effects are associated, so the most deleterious effect is chosen
                    elif len(effect) > 1:
                        effects_temp.append(rank_effects(effect))

            # concatenates elements of effects_temp into one string and stores it in effects
            effects.append(','.join(effects_temp))
    
    return effects

The **rank_effects** function iterates through a list of all effects caused by a single allele and returns the most deleterious effect. The first if statement contains the most deleterious effect, and following if statements are ranked less deleterious than the previous.

In [916]:
# returns most deleterious effect if variant has two or more effects
def rank_effects(effects):
    if 'stop_gained' in effects:
        return 'stop_gained'
    elif 'stop_lost' in effects:
        return 'stop_lost'
    elif 'missense_variant' in effects:
        return 'missense_variant'
    elif 'initiator_codon_variant' in effects:
        return 'initiator_codon_variant'
    elif 'splice_acceptor_variant' in effects:
        return 'splice_acceptor_variant'
    elif 'splice_donor_variant' in effects:
        return 'splice_donor_variant'
    elif 'splice_region_variant' in effects:
        return 'splice_region_variant'
    elif '5_prime_UTR_variant' in effects:
        return '5_prime_UTR_variant'
    elif '3_prime_UTR_variant' in effects:
        return '3_prime_UTR_variant'
    elif 'non_coding_transcript_exon_variant' in effects:
        return 'non_coding_transcript_exon_variant'
    elif 'intron_variant' in effects:
        return 'intron_variant'
    elif 'stop_retained_variant' in effects:
        return 'stop_retained_variant'
    elif 'synonymous_variant' in effects:
        return 'synonymous_variant'
    else:
        return '.'

Let's run **effects**. I've printed three results here. The first result is a null value, which means this variant does not have an effect listed in the ExAC database. The second result does have an effect, missense_variant. The third result has two alleles, but neither exist in the ExAC database.

In [917]:
consequences = effects(variants)
print(consequences[0])
print(consequences[1])
print(consequences[53])

.
missense_variant
.,.


The **reads_stats** function returns the total read depth, the number of reads supporting the variant, and the percentage of reads supporting the variant across all samples.

In [919]:
def reads_stats(variants):
    stats = []
    
    for var in variants:
        DP = var.info['DP'] # read depth
        AOs = var.info['AO'] # allele observation count

        # one allele associated with variant
        if len(AOs) == 1:
            AO = AOs[0]
            percent = round((AO/DP)*100, 2)
            stats.append([str(DP), str(AO), str(percent)])

        # more than one allele associated 
        elif len(AOs) > 1:
            percents = []
            
            # iterates through each allele 
            for AO in AOs:
                percent = round((AO/DP)*100, 2)
                percents.append(str(percent))
                
            AOs = [str(AO) for AO in AOs] 
            stats.append([str(DP), ','.join(AOs), ','.join(percents)])
    
    return stats

Let's run **read_stats**. The function returns a list of lists. In the first result is a list containing, from left to right, the total read depth, the number of reads supporting the variant, and the percent of reads supporting the variant. In this case, it'd be 4124 total reads, 95 reads support the variant, so 2.30% of reads support the variant. 

In the second result, this variant has alleles. To comprehend this, there are 2232 total reads, 70 reads support the first allele of this variant while 30 reads support the second allele of this variant. Therefore, 3.14% of reads support the first allele and 1.34% of reads support the second allele.

In [920]:
reads = reads_stats(variants)
print(reads[0])
print(reads[53])

['4124', '95', '2.3']
['2232', '70,30', '3.14,1.34']


The **allele_freq** function returns the allele frequency of each variant. The allele frequency is fetched from the ExAC API. Variants with no known allele frequency from ExAC are denoted with a '.' as a null value.

In [926]:
def allele_freq(variants):
    AFs = []
    
    for var in variants:
        chrom = str(var.chrom)
        pos = str(var.pos)
        ref = var.ref
        alts = var.alts

        # one allele associated with variant
        if len(alts) == 1:

            # ExAC API request
            var_id = f'{chrom}-{pos}-{ref}-{alts[0]}'
            request = f'http://exac.hms.harvard.edu/rest/variant/variant/{var_id}'
            response = requests.get(request)
            text = json.loads(response.text)
            
            if 'allele_freq' in text.keys():
                AF = text['allele_freq']
                AFs.append(str(AF))   
            else:
                AFs.append('.')

        # more than one alleles associated with variant
        if len(alts) > 1:
            AFs_temp = []
            
            # iterate through each allele
            for alt in alts:

                # ExAC API request
                var_id = f'{chrom}-{pos}-{ref}-{alt}'
                request = f'http://exac.hms.harvard.edu/rest/variant/variant/{var_id}'
                response = requests.get(request)
                text = json.loads(response.text)
                
                if 'allele_freq' in text.keys():
                    AF = text['allele_freq']
                    AFs_temp.append(str(AF))   
                else:
                    AFs_temp.append('.')

            # concatenates elements of AFs_temp into one string and stores it in AFs
            AFs.append(','.join(AFs_temp))

    return AFs

Let's run **allele_freq**. Like **effects**, results are fetched from the ExAC API. Here, the first result does not have an allele frequency in the ExAC database, so the null value is denoted as '.'. The second result does have an allele frequency, 0.6611108268522309. The third result has two alleles for this variant, but neither are found in the ExAC database.

In [927]:
AFs = allele_freq(variants)
print(AFs[0])
print(AFs[1])
print(AFs[53])

.
0.6611108268522309
.,.


The **reads_per_sample_stats** function is very similar to the **reads_stats** function. However, it calculates the same information for each sample i.e. read depth for sample1, sample2, etc.

In [925]:
def reads_per_sample_stats(variants):
    reads_per_sample_stats = []
    
    for var in variants:
        temp = []

        #iterates through each sample
        for sample in var.samples:
            DP = var.samples[sample]['DP'] # sample read depth
            AOs = var.samples[sample]['AO'] # sample allele observatopm count(s)

            # one allele associated with variant
            if len(AOs) == 1:
                AO = AOs[0]
                percent = round((AO/DP)*100, 2)
                temp.append([str(DP), str(AO), str(percent)])

            # more than one allele associated
            elif len(AOs) > 1:
                percents = []
                
                # iterates through each allele
                for AO in AOs:
                    percent = round((AO/DP)*100, 2)
                    percents.append(str(percent))
                    
                AOs = [str(AO) for AO in AOs] 
                temp.append([str(DP), ','.join(AOs), ','.join(percents)])

        # appends list of samples to reads_per_sample_stats
        reads_per_sample_stats.append(temp)

    return reads_per_sample_stats

Let's run **reads_per_sample_stats**. This function returns a list of lists. Within each element(list) are lists containing the same information as **reads_stats**, but for each sample i.e each list corresponds to each sample. This is done by zipping together the odd indexes of reads_per_sample_stats with the even indexes.

For example, in the first result, this list has two lists because there are two samples in the VCF file: "normal" and "vaf5". For this list, the first element corresponds to the first sample, and so on. For this particular variant, the "normal" sample has 2063 reads, but 0 reads support the variant. The "vaf5" sample has 2061 reads, and 95 of those reads support the variant, so 4.61% reads support the variant just within the "vaf5" sample.

In [898]:
reads_per_sample = reads_per_sample_stats(variants)
print(reads_per_sample[0])
print(reads_per_sample[53])

Reads per sample stats annotated.
[['2063', '0', '0.0'], ['2061', '95', '4.61']]
[['1116', '35,15', '3.14,1.34'], ['1116', '35,15', '3.14,1.34']]


### Writing TSV file
Now we can finally begin to write the output file. First, we must construct the header.

In [928]:
columns = ['ref', 'alt', 'variant_type', 'effect','total_read_depth', 'total_alt_allele_reads', 
               'percent_total_alt_allele_reads(%)', 'allele_frequency']
    
for sample in samples:
    columns.append(f'{sample}_read_depth')
    columns.append(f'{sample}_alt_allele_reads')
    columns.append(f'{sample}_percent_total_alt_allele_reads(%)')

header = '\t'.join(columns)
print(header)

ref	alt	variant_type	effect	total_read_depth	total_alt_allele_reads	percent_total_alt_allele_reads(%)	allele_frequency	normal_read_depth	normal_alt_allele_reads	normal_percent_total_alt_allele_reads(%)	vaf5_read_depth	vaf5_alt_allele_reads	vaf5_percent_total_alt_allele_reads(%)


Next, we open a tsv file called challenge_data_output.tsv. 

We then create a list of integers that are a range of the number of rows in the input VCF file. Now, we iterate through all of the lists that were returned from the annotation functions and write it into the file. Finally, we close the file.

In [929]:
output = open('challenge_data_output_100.tsv', 'w')
output.write(header+'\n')

index = range(len(records))

for i in index:
    row = '\t'.join([variants[i].ref, ','.join(variants[i].alts), var_types[i], consequences[i], \
          '\t'.join(reads[i]), AFs[i]]) + '\t'

    for j in range(len(samples)):
        row = row + '\t'.join(reads_per_sample[i][j]) + '\t'
        row.strip('\t')

    output.write(row + '\n')

output.close()

We can take a look at how the output file looks like here:

In [930]:
import pandas as pd
tsv = pd.read_csv('challenge_data_output_100.tsv', delimiter='\t', index_col=False)
tsv.head(5)

,ref,alt,variant_type,effect,total_read_depth,total_alt_allele_reads,percent_total_alt_allele_reads(%),allele_frequency,normal_read_depth,normal_alt_allele_reads,normal_percent_total_alt_allele_reads(%),vaf5_read_depth,vaf5_alt_allele_reads,vaf5_percent_total_alt_allele_reads(%)
0,G,T,snp,.,4124,95,2.3,.,2063,0,0.0,2061,95,4.61
1,C,A,snp,missense_variant,1134,652,57.5,0.6611108268522309,567,326,57.5,567,326,57.5
2,T,C,snp,non_coding_transcript_exon_variant,786,786,100.0,0.997571146075144,393,393,100.0,393,393,100.0
3,G,A,snp,5_prime_UTR_variant,228,228,100.0,0.8972993695689214,114,114,100.0,114,114,100.0
4,G,A,snp,splice_region_variant,4055,94,2.32,1.1137593834228053e-05,2057,0,0.0,1998,94,4.7


This concludes the extended guide to this prototype variant annotation tool.